# Cash Flow Forecasting với LSTM

Notebook này sẽ hướng dẫn cách sử dụng pipeline LSTM để dự đoán dòng tiền của ngân hàng.

## Mục lục
1. [Cài đặt thư viện](#1-cài-đặt-thư-viện)
2. [Import pipeline](#2-import-pipeline)
3. [Chạy pipeline hoàn chỉnh](#3-chạy-pipeline-hoàn-chỉnh)
4. [Phân tích kết quả chi tiết](#4-phân-tích-kết-quả-chi-tiết)
5. [Tùy chỉnh dự đoán](#5-tùy-chỉnh-dự-đoán)

## 1. Cài đặt thư viện

Chạy lệnh sau để cài đặt các thư viện cần thiết:

In [ ]:
# Cài đặt requirements
!pip install -r requirements.txt

## 2. Import pipeline

In [ ]:
# Import pipeline
from cash_flow_lstm_pipeline import CashFlowLSTMPipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Pipeline đã được import thành công!")

## 3. Chạy pipeline hoàn chỉnh

Pipeline sẽ tự động:
- Tải và xử lý dữ liệu từ tất cả các file CSV
- Tạo features engineering
- Training mô hình LSTM
- Đánh giá mô hình
- Dự đoán 30 ngày tới
- Tạo biểu đồ và lưu kết quả

In [ ]:
# Khởi tạo pipeline
data_path = "./"  # Đường dẫn hiện tại chứa các file CSV
pipeline = CashFlowLSTMPipeline(data_path)

# Chạy pipeline hoàn chỉnh
print("Bắt đầu chạy pipeline...")
results = pipeline.run_complete_pipeline(days_ahead=30)

print("\n🎉 Pipeline hoàn thành!")

## 4. Phân tích kết quả chi tiết

In [ ]:
# Hiển thị metrics đánh giá mô hình
print("📊 METRICS ĐÁNH GIÁ MÔ HÌNH:")
print("=" * 40)
for metric, value in results['metrics'].items():
    if metric == 'MAPE':
        print(f"{metric:8s}: {value:8.2f}%")
    else:
        print(f"{metric:8s}: {value:8.4f}")

# Đánh giá chất lượng mô hình
r2 = results['metrics']['R²']
mape = results['metrics']['MAPE']

print("\n🎯 ĐÁNH GIÁ CHẤT LƯỢNG:")
if r2 > 0.8:
    print(f"✅ Mô hình rất tốt (R² = {r2:.3f})")
elif r2 > 0.6:
    print(f"✅ Mô hình tốt (R² = {r2:.3f})")
elif r2 > 0.4:
    print(f"⚠️ Mô hình trung bình (R² = {r2:.3f})")
else:
    print(f"❌ Mô hình cần cải thiện (R² = {r2:.3f})")

if mape < 10:
    print(f"✅ Độ chính xác cao (MAPE = {mape:.1f}%)")
elif mape < 20:
    print(f"✅ Độ chính xác tốt (MAPE = {mape:.1f}%)")
elif mape < 30:
    print(f"⚠️ Độ chính xác trung bình (MAPE = {mape:.1f}%)")
else:
    print(f"❌ Độ chính xác thấp (MAPE = {mape:.1f}%)")

In [ ]:
# Hiển thị thống kê dữ liệu
print("📈 THỐNG KÊ DỮ LIỆU DÒNG TIỀN:")
print("=" * 50)

cash_flow_stats = results['cash_flow_data']['net_cash_flow'].describe()
print(f"Số ngày dữ liệu: {len(results['cash_flow_data'])}")
print(f"Dòng tiền trung bình/ngày: {cash_flow_stats['mean']:,.0f}")
print(f"Dòng tiền cao nhất: {cash_flow_stats['max']:,.0f}")
print(f"Dòng tiền thấp nhất: {cash_flow_stats['min']:,.0f}")
print(f"Độ lệch chuẩn: {cash_flow_stats['std']:,.0f}")

# Phân tích xu hướng
positive_days = (results['cash_flow_data']['net_cash_flow'] > 0).sum()
negative_days = (results['cash_flow_data']['net_cash_flow'] < 0).sum()
print(f"\nSố ngày dòng tiền dương: {positive_days} ({positive_days/len(results['cash_flow_data'])*100:.1f}%)")
print(f"Số ngày dòng tiền âm: {negative_days} ({negative_days/len(results['cash_flow_data'])*100:.1f}%)")

In [ ]:
# Hiển thị dự đoán tương lai
print("🔮 DỰ ĐOÁN DÒNG TIỀN 30 NGÀY TỚI:")
print("=" * 60)

future_df = results['future_predictions']

# Hiển thị 10 ngày đầu
print("10 ngày đầu:")
for i, (_, row) in enumerate(future_df.head(10).iterrows()):
    date_str = row['date'].strftime('%Y-%m-%d (%A)')
    cash_flow = row['predicted_cash_flow']
    cumulative = row['predicted_cumulative_cash_flow']
    
    status = "📈" if cash_flow > 0 else "📉"
    print(f"{status} {date_str}: {cash_flow:>10,.0f} (Tích lũy: {cumulative:>12,.0f})")

# Thống kê dự đoán
print("\n📊 THỐNG KÊ DỰ ĐOÁN:")
future_stats = future_df['predicted_cash_flow'].describe()
print(f"Dòng tiền trung bình/ngày: {future_stats['mean']:,.0f}")
print(f"Dòng tiền cao nhất: {future_stats['max']:,.0f}")
print(f"Dòng tiền thấp nhất: {future_stats['min']:,.0f}")

# Dự đoán rủi ro
negative_future_days = (future_df['predicted_cash_flow'] < 0).sum()
if negative_future_days > 0:
    print(f"\n⚠️ CẢNH BÁO: Có {negative_future_days} ngày dự kiến dòng tiền âm")
else:
    print(f"\n✅ Dòng tiền dự kiến luôn dương trong 30 ngày tới")

## 5. Tùy chỉnh dự đoán

Bạn có thể thay đổi số ngày dự đoán và chạy lại:

In [ ]:
# Dự đoán cho 60 ngày
print("🔮 Dự đoán cho 60 ngày tới...")
future_60_days = pipeline.predict_future(days_ahead=60)

# Vẽ biểu đồ dự đoán 60 ngày
plt.figure(figsize=(15, 8))

# Subplot 1: Daily predictions
plt.subplot(2, 1, 1)
plt.plot(future_60_days['date'], future_60_days['predicted_cash_flow'], 
         color='red', marker='o', markersize=3, alpha=0.7)
plt.title('Dự đoán dòng tiền hàng ngày - 60 ngày tới', fontsize=14, fontweight='bold')
plt.ylabel('Dòng tiền (VND)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Subplot 2: Cumulative predictions
plt.subplot(2, 1, 2)
plt.plot(future_60_days['date'], future_60_days['predicted_cumulative_cash_flow'], 
         color='purple', linewidth=2, alpha=0.8)
plt.title('Dự đoán dòng tiền tích lũy - 60 ngày tới', fontsize=14, fontweight='bold')
plt.xlabel('Ngày')
plt.ylabel('Dòng tiền tích lũy (VND)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# Lưu dự đoán 60 ngày
future_60_days.to_csv('future_60_days_predictions.csv', index=False)
print("✅ Đã lưu dự đoán 60 ngày vào file 'future_60_days_predictions.csv'")

In [ ]:
# Phân tích theo tuần
print("📅 PHÂN TÍCH DỰ ĐOÁN THEO TUẦN:")
print("=" * 50)

future_60_days['week'] = future_60_days['date'].dt.isocalendar().week
weekly_analysis = future_60_days.groupby('week').agg({
    'predicted_cash_flow': ['sum', 'mean', 'std'],
    'date': ['min', 'max']
}).round(0)

weekly_analysis.columns = ['Total', 'Average', 'StdDev', 'Start_Date', 'End_Date']

for week, row in weekly_analysis.iterrows():
    status = "📈" if row['Total'] > 0 else "📉"
    print(f"{status} Tuần {week}: Tổng = {row['Total']:>10,.0f}, TB = {row['Average']:>8,.0f}")

## 🎯 Kết luận

Pipeline LSTM đã được triển khai thành công với các tính năng:

### ✅ Đã hoàn thành:
- Tích hợp tất cả 6 nguồn dữ liệu (deposits, loans, withdrawals, bond sales, operating expenses, interbank transfers)
- Features engineering với đặc trưng thời gian và moving averages
- Mô hình LSTM với 3 layers và regularization
- Đánh giá mô hình với multiple metrics
- Dự đoán tương lai với khả năng tùy chỉnh
- Visualization và báo cáo tự động

### 📊 Các file kết quả:
- `model_evaluation_metrics.csv`: Metrics đánh giá mô hình
- `future_cash_flow_predictions.csv`: Dự đoán 30 ngày
- `processed_cash_flow_data.csv`: Dữ liệu đã xử lý
- `cash_flow_analysis_results.png`: Biểu đồ phân tích

### 🚀 Ứng dụng thực tế:
- Quản lý thanh khoản ngân hàng
- Dự đoán nhu cầu vốn
- Cảnh báo rủi ro dòng tiền
- Hỗ trợ quyết định đầu tư